In [29]:
import base64
from openai import OpenAI
import time

client = OpenAI()

In [7]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_path = "test1.png"
base64_image = encode_image(image_path)

In [21]:
file = client.files.create(
    file=open("test1.png", "rb"),
    purpose="assistants",
)

# GPT-4.1

In [13]:
response = client.responses.create(
    model="gpt-4.1",
    input=[{
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"
            },
            {
                "type": "input_image",
                "image_url": f"data:image/png;base64,{base64_image}"
            },
        ],
    }],
)

print(response.output_text)

以下は、グラフから**hoge**（青線）の各年度推定値を有効数字8桁でカンマ区切りで出力したものです。

```
0.80000000,0.50000000,0.60000000,0.80000000,0.90000000,0.40000000,0.30000000,0.90000000,0.20000000,0.40000000,0.80000000,0.90000000,0.70000000,0.50000000,0.80000000,0.40000000,0.90000000,0.50000000,0.50000000,0.20000000,0.70000000,0.30000000,0.70000000,0.80000000,0.30000000,0.40000000
```

年ごとの並びは左から2000年～2025年です。  
グラフから読み取った値のため、おおよその推測値です。


In [34]:
assistant = client.beta.assistants.create(
    model="gpt-4.1",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id],
        },
    },
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=[
        {"type": "image_file", "image_file": {"file_id": file.id}},
        {"type": "text", "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"},
    ],
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    time.sleep(10)

messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc",
)

for message in messages:
    if message.content[0].type == "text":
        print(message.role, ":", message.content[0].text.value)

user : file-RexN9ByMdhGExHu1v85kJ3
assistant : グラフから「hoge」の数値を推測して、各年度分を有効数字8桁でカンマ区切りで出力します。  
※目視による概算である旨ご容赦ください。

2000: 0.80  
2001: 1.00  
2002: 0.60  
2003: 0.90  
2004: 0.70  
2005: 1.00  
2006: 0.10  
2007: 0.00  
2008: 0.20  
2009: 0.20  
2010: 0.10  
2011: 0.20  
2012: 0.50  
2013: 0.50  
2014: 0.70  
2015: 0.80  
2016: 0.40  
2017: 0.60  
2018: 0.30  
2019: 1.00  
2020: 0.60  
2021: 1.00  
2022: 0.40  
2023: 0.80  
2024: 0.50

**カンマ区切り版（8桁有効数字、小数点制約上ゼロ埋め含む）：**  
0.80000000,1.00000000,0.60000000,0.90000000,0.70000000,1.00000000,0.10000000,0.00000000,0.20000000,0.20000000,0.10000000,0.20000000,0.50000000,0.50000000,0.70000000,0.80000000,0.40000000,0.60000000,0.30000000,1.00000000,0.60000000,1.00000000,0.40000000,0.80000000,0.50000000

もし他の精度や形式で出力したい場合はご指示ください。


# o4-mini

In [35]:
response = client.responses.create(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"
            },
            {
                "type": "input_image",
                "image_url": f"data:image/png;base64,{base64_image}"
            },
        ],
    }],
)

print(response.output_text)

0.60000000,0.65000000,0.20000000,0.78000000,0.82000000,0.95000000,0.92000000,0.30000000,0.28000000,0.82000000,0.95000000,0.60000000,0.70000000,0.65000000,0.60000000,0.65000000,0.55000000,0.53000000,0.75000000,0.60000000,0.55000000,0.60000000,0.12000000,0.88000000,0.25000000,0.88000000


# o1

In [18]:
response = client.responses.create(
    model="o1",
    input=[{
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"
            },
            {
                "type": "input_image",
                "image_url": f"data:image/png;base64,{base64_image}"
            },
        ],
    }],
)

print(response.output_text)

0.70000000,0.50000000,0.60000000,0.80000000,0.30000000,0.40000000,0.70000000,0.50000000,0.30000000,0.90000000,0.40000000,0.70000000,0.80000000,0.50000000,0.20000000,0.90000000,0.70000000,0.60000000,0.20000000,0.80000000,0.60000000,0.30000000,1.00000000,0.50000000,0.60000000,0.70000000


# GPT-4o

In [20]:
response = client.responses.create(
    model="gpt-4o",
    input=[{
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"
            },
            {
                "type": "input_image",
                "image_url": f"data:image/png;base64,{base64_image}"
            },
        ],
    }],
)

print(response.output_text)

申し訳ありませんが、画像から数値を正確に推測することはできません。ただし、グラフを詳細に確認したり、より高解像度のデータを使うことで、より正確に数値を推定できるかもしれません。テキストデータなどがあればお手伝いできることがあります。


In [40]:
assistant = client.beta.assistants.create(
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id],
        },
    },
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=[
        {"type": "image_file", "image_file": {"file_id": file.id}},
        {"type": "text", "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"},
    ],
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    time.sleep(10)

messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc",
)

for message in messages:
    if message.content[0].type == "text":
        print(message.role, ":", message.content[0].text.value)

assistant : 画像の時系列グラフからhogeの数値を推測して年度ごとの値を有効数字8桁でカンマ区切りで出力します。ただし、視覚的な観察に基づく推定になるため、正確な数値ではなく参考程度の値としてお考えください。

各データポイントの推定は全体の傾向と数値の変化率を考慮して行います。2021年から2000年までの推定値を始めます。
assistant : 先ほどの説明では間違いがありました。もう一度グラフを読み取り、2000年から2025年までの順序での推定値を出力します。しばらくお待ちください。
assistant : ここで推定される2000年から2025年までのhogeの数値を示します：

- 2000: 0.95
- 2001: 0.30
- 2002: 0.92
- 2003: 0.60
- 2004: 0.95
- 2005: 0.30
- 2006: 0.90
- 2007: 0.65
- 2008: 0.05
- 2009: 0.95
- 2010: 0.60
- 2011: 0.50
- 2012: 0.70
- 2013: 0.30
- 2014: 0.90
- 2015: 0.50
- 2016: 0.10
- 2017: 0.95
- 2018: 0.10
- 2019: 0.40
- 2020: 0.45
- 2021: 0.95
- 2022: 0.80
- 2023: 0.90
- 2024: 0.70
- 2025: 0.45

それでは、これらの推定を有効数字8桁でカンマ区切りで出力します。
assistant : こちらが2000年から2025年までの推定値を有効数字8桁でカンマ区切りにしたものです：

```
0.95,0.3,0.92,0.6,0.95,0.3,0.9,0.65,0.05,0.95,0.6,0.5,0.7,0.3,0.9,0.5,0.1,0.95,0.1,0.4,0.45,0.95,0.8,0.9,0.7,0.45
```

この数値は視覚的にグラフを読み取って推測された値であり、正確性は保証できません。参考としてお使いください。


# GPT-4.5-Preview

In [41]:
response = client.responses.create(
    model="gpt-4.5-preview",
    input=[{
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"
            },
            {
                "type": "input_image",
                "image_url": f"data:image/png;base64,{base64_image}"
            },
        ],
    }],
)

print(response.output_text)

2000,0.6000000  
2001,0.1500000  
2002,0.8500000  
2003,0.4000000  
2004,0.2000000  
2005,0.9500000  
2006,0.3000000  
2007,0.3500000  
2008,0.7000000  
2009,0.9000000  
2010,0.4000000  
2011,0.7500000  
2012,0.5500000  
2013,0.8000000  
2014,0.5500000  
2015,0.8500000  
2016,0.2500000  
2017,0.4000000  
2018,0.8000000  
2019,0.2000000  
2020,0.6000000  
2021,0.8500000  
2022,0.3000000  
2023,0.5500000  
2024,0.2000000  
2025,0.7500000


In [42]:
assistant = client.beta.assistants.create(
    model="gpt-4.5-preview",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id],
        },
    },
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=[
        {"type": "image_file", "image_file": {"file_id": file.id}},
        {"type": "text", "text": "グラフからhogeの数値を推測して各年度分有効数字8桁でカンマ区切りで出力してください。"},
    ],
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    time.sleep(10)

messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc",
)

for message in messages:
    if message.content[0].type == "text":
        print(message.role, ":", message.content[0].text.value)

assistant : グラフから各年度の数値を読み取って、推測値を以下のように示します。

2000: 0.9  
2001: 0.2  
2002: 0.9  
2003: 0.75  
2004: 0.03  
2005: 0.95  
2006: 0.65  
2007: 0.05  
2008: 0.9  
2009: 0.25  
2010: 0.75  
2011: 0.35  
2012: 0.02  
2013: 0.05  
2014: 0.45  
2015: 0.65  
2016: 0.25  
2017: 0.85  
2018: 0.35  
2019: 0.7  
2020: 0.05  
2021: 1.0  
2022: 0.3  
2023: 0.95  
2024: 0.1  
2025: 0.8  

これらを有効数字8桁でカンマ区切りで表現すると以下のようになります。

0.90000000,0.20000000,0.90000000,0.75000000,0.03000000,0.95000000,0.65000000,0.05000000,0.90000000,0.25000000,0.75000000,0.35000000,0.02000000,0.05000000,0.45000000,0.65000000,0.25000000,0.85000000,0.35000000,0.70000000,0.05000000,1.00000000,0.30000000,0.95000000,0.10000000,0.80000000
